# CAR-19 Data Analysis Goal 4: Confirm charge accumulation rate

This notebook runs the code in the nircam_calib/comissioning/NRC_19_subarrays module in order to analyze data from CAR-19 (Subarray Verification)

## Table of Contents

* [Goal 4: Confirm same charge accumulation rate in subarrays vs full frame](#goal_4)

In [ ]:
from glob import glob
import os

In [ ]:
from astropy.io import ascii, fits
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
from jwst.pipeline.calwebb_image2 import Image2Pipeline
from jwst.pipeline.calwebb_image3 import Image3Pipeline
from jwst.pipeline.calwebb_tso3 import Tso3Pipeline
from jwst.source_catalog import SourceCatalogStep
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from nircam_calib.commissioning.NRC_19_subarrays.subarray_loc_examination_prep import check_location
from nircam_calib.commissioning.NRC_19_subarrays import confirm_subarray_location_via_sources as locations
from nircam_calib.commissioning.NRC_19_subarrays import confirm_telescope_pointing as pointing
from nircam_calib.commissioning.NRC_19_subarrays import confirm_count_rates as count_rates
from nircam_calib.commissioning.utils.display import overlay_catalog

In [ ]:
import jwst
print(jwst.__version__)

In [ ]:
base_dir = '/ifs/jwst/wit/nircam/simulationWG/Imaging/CAR-19/2021_June'
#base_dir = 'path_to_simulationWG_CAR-19_directory'

In [ ]:
def display(files, titles, minval=0, maxval=10):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 10))

    for ax, filename, title in zip(axes.flat, files, titles):
        img = fits.getdata(filename)
        ax.set_axis_off()
        im = ax.imshow(img, origin='lower', cmap='viridis', vmin=minval, vmax=maxval)
        ax.set_title(title)

    cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.5)
    cbar.set_ticks(np.linspace(minval, maxval, 5))
    plt.show()

<a id='goal_4'></a>
## Goal 4: Charge Accumulation Rate

Show that the charge accumulation rate in the subarray files is consistent with that in the full frame data

First, check LW data using calibrated slope images

### Full frame versus SUB160

#### Longwave

In [ ]:
#fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits'
fullframe_file = 'Pipeline_Level3/jw01068004001_01101_00001_nrcb5_a3001_crf.fits'
#fullframe_file = 'Pipeline_Level1/jw01068004001_01101_00001_nrcb5_assignwcsstep.fits'

In [ ]:
#subfiles = sorted(glob('Pipeline_Level2/jw01068001*nrcb5_cal.fits'))
subfiles = sorted(glob('Pipeline_Level3/jw01068001*nrcb5*_crf.fits'))
#subfiles = sorted(glob('Pipeline_Level1/jw01068001*nrcb5_assignwcsstep.fits'))
print('subfiles:')
for s in subfiles:
    print(s)

In [ ]:
# Rate files that match the cal files above. This is for a test to see if the 
# discrepancy in photometry results between the LW full frame and subarray
# data is also present in the rate files.
fullframe_rate_file = 'Pipeline_Level1/jw01068004001_01101_00001_nrcb5_rate.fits'
subratefiles = sorted(glob('Pipeline_Level1/jw01068001*nrcb5_rate.fits'))
print(fullframe_rate_file)
for s in subratefiles:
    print(s)

In [ ]:
# sub_dq and full_dq columns: if True, then there is at least one non-zero value
# in the DQ array within the photometry aperture in the subarray or full frame
# images, respectively. In some cases this may not matter. But in some cases the
# flagged pixel defintely has an effect.

# d_phot_p is the percentage difference in the photometry between the subarray
# and full frame cases. Eqn is (full frame - subarray) / full frame, so negative
# means the source is brighter in the subarray data


sub_thresh = 30    # n-sigma source detection threshold for subarray input file
full_thresh = 200  # n-sigma source detection threshold for full frame input file
max_separation = 2 # Maximum allowed separation (pixels) between sources in subarray vs full-frame
aperture = 5       # Aperture photometry radius (pixels)
bkgd_inner = 6     # Aperture photometry inner radius for background (pixels)
bkgd_outer = 8     # Aperture photometry outer radius for background (pixels)

# ORIGINAL CALL - THE DISCREPANCY IN PHOTOMETRY IS THE SAME REGARDLESS OF WHETHER
# YOU USE CAL OR CRF FILES.
#for subarray_file in subfiles:
#    print('Trying: {}'.format(subarray_file))
#    count_rates.compare_rate_images(subarray_file, fullframe_file,
#                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
#                                   max_separation=max_separation, aperture_radius=aperture,
#                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)
    
# MODIFIED CALL, TESTING TO SEE IF DISCREPANCY IN PHOTOMETRY IS PRESENT IN RATE FILES
# RESULTS: THE DISCREPANCY IS STILL THERE...
for subarray_file, subratefile in zip(subfiles, subratefiles):
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer,
                                   subarray_rate_file=subratefile, fullframe_rate_file=fullframe_rate_file)

#### Shortwave

Now look at SW data calibrated slope images

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb1_cal.fits'

In [ ]:
subfiles = sorted(glob('Pipeline_Level2/jw01068001*nrcb[1234]_cal.fits'))
print('subfiles:')
for s in subfiles:
    print(s)

In [ ]:
sub_thresh = 50
full_thresh = 200
max_separation = 2
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb2_cal.fits'
sub_thresh = 10
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb3_cal.fits'
sub_thresh = 50
full_thresh = 200
max_separation = 3.
aperture = 5
bkgd_inner = 6
bkgd_outer = 8

for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb4_cal.fits'
sub_thresh = 50
full_thresh = 200
max_separation = 1
aperture = 5
bkgd_inner = 6
bkgd_outer = 8

for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

### Full frame versus SUB320

#### Longwave

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits'
subfiles = sorted(glob('Pipeline_Level2/jw01068002*nrcb5_cal.fits'))
sub_thresh = 50
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8

for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

#### Shortwave

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb1_cal.fits'
subfiles = sorted(glob('Pipeline_Level2/jw01068002*nrcb[1234]_cal.fits'))
print('subfiles:')
for s in subfiles:
    print(s)

In [ ]:
sub_thresh = 30
full_thresh = 200
max_separation = 2
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb2_cal.fits'
sub_thresh = 100
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb3_cal.fits'
sub_thresh = 30
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb4_cal.fits'
sub_thresh = 30
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

### Full frame versus SUB640

#### Longwave

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits'
subfiles = sorted(glob('Pipeline_Level2/jw01068003*nrcb5_cal.fits'))
sub_thresh = 150
full_thresh = 200
max_separation = 1
aperture = 5
bkgd_inner = 6
bkgd_outer = 8

for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

#### Shortwave

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb1_cal.fits'
subfiles = sorted(glob('Pipeline_Level2/jw01068003*nrcb[1234]_cal.fits'))
print('subfiles:')
for s in subfiles:
    print(s)

In [ ]:
sub_thresh = 100
full_thresh = 200
max_separation = 2
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb2_cal.fits'
sub_thresh = 30
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb3_cal.fits'
sub_thresh = 30
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb4_cal.fits'
sub_thresh = 30
full_thresh = 200
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

#### Compare two differently sized subarrays to check if the photomety results are comparable 

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068003001_01101_00001_nrcb4_cal.fits'
subfiles = sorted(glob('Pipeline_Level2/jw01068002*nrcb4_cal.fits'))
sub_thresh = 30
full_thresh = 30
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles[0:1]:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level2/jw01068003001_01101_00001_nrcb5_cal.fits'
subfiles = sorted(glob('Pipeline_Level2/jw01068002*nrcb5_cal.fits'))
sub_thresh = 30
full_thresh = 30
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

#### Try running on i2d files instead

In [ ]:
fullframe_file = 'Pipeline_Level3/full_sw_i2d.fits'
subfiles = sorted(glob('Pipeline_Level3/sub[1346]*_sw_i2d.fits'))
sub_thresh = 100
full_thresh = 500
max_separation = 3
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

In [ ]:
fullframe_file = 'Pipeline_Level3/full_lw_i2d.fits'
subfiles = sorted(glob('Pipeline_Level3/sub[1346]*_lw_i2d.fits'))
sub_thresh = 50
full_thresh = 500
max_separation = 0.5
aperture = 5
bkgd_inner = 6
bkgd_outer = 8
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    count_rates.compare_rate_images(subarray_file, fullframe_file,
                                   subarray_threshold=sub_thresh, fullframe_threshold=full_thresh,
                                   max_separation=max_separation, aperture_radius=aperture,
                                   bkgd_in_radius=bkgd_inner, bkgd_out_radius=bkgd_outer)

<h3>Next, compare data in the level 3 output source catalogs</h3>

In [ ]:
# Try running the source_catalog step on the cal files for the 
# time series observations. The source catalog that is produced by
# default for time series data is not the same format as that for the
# other observations, and so the two cannot be easily compared. 
cal_files = ['Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits',
             'Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits',
             'Pipeline_Level2/jw01068006001_01101_00001_nrcb1_cal.fits',
             'Pipeline_Level2/jw01068006001_01101_00002_nrcb5_cal.fits',
             'Pipeline_Level2/jw01068007001_01101_00001-seg001_nrca3_cal.fits']

In [ ]:
for cfile in cal_files:
    sc = SourceCatalogStep.call(cfile, save_results=True, output_dir='Pipeline_Level3')

In [ ]:
SourceCatalogStep.call('Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits')

In [ ]:
catalog_directory = '/ifs/jwst/wit/nircam/simulationWG/Imaging/CAR-19/2021_June/Pipeline_Level3'
sw_full_cat = os.path.join(catalog_directory, 'full_sw_cat.ecsv')
lw_full_cat = os.path.join(catalog_directory, 'full_lw_cat.ecsv')
sw_160_cat = os.path.join(catalog_directory, 'sub160_sw_cat.ecsv')
lw_160_cat = os.path.join(catalog_directory, 'sub160_lw_cat.ecsv')
sw_320_cat = os.path.join(catalog_directory, 'sub320_sw_cat.ecsv')
lw_320_cat = os.path.join(catalog_directory, 'sub320_lw_cat.ecsv')
sw_640_cat = os.path.join(catalog_directory, 'sub640_sw_cat.ecsv')
lw_640_cat = os.path.join(catalog_directory, 'sub640_lw_cat.ecsv')
sw_400p_cat = os.path.join(catalog_directory, 'sub400p_sw_phot.ecsv')
lw_400p_cat = os.path.join(catalog_directory, 'sub400p_lw_phot.ecsv')
sw_64p_cat = os.path.join(catalog_directory, 'sub64p_sw_phot.ecsv')
lw_64p_cat = os.path.join(catalog_directory, 'sub64p_lw_phot.ecsv')
sw_subarray_cats = [sw_160_cat, sw_320_cat, sw_640_cat] #, sw_400p_cat, sw_64p_cat]
lw_subarray_cats = [lw_160_cat, lw_320_cat, lw_640_cat] #, lw_400p_cat, lw_64p_cat]

#### Display i2d files and catalogs, to get an idea of how source finding went

In [ ]:
swfull_i2d = fits.getdata('Pipeline_Level3/full_sw_i2d.fits')
swfull_catalog = ascii.read(sw_full_cat)

In [ ]:
# There are so many sources found that the entire field of view turns red.
overlay_catalog(swfull_i2d, swfull_catalog, vmin=2, vmax=100)

In [ ]:
sw160_i2d = fits.getdata('Pipeline_Level3/sub160_sw_i2d.fits')
sw160_catalog = ascii.read(sw_160_cat)

In [ ]:
# All sources found, but lots of junk as well.
overlay_catalog(sw160_i2d, sw160_catalog, vmin=2, vmax=100)

### Try running SourceCatalogStep on the i2d files with better parameter values

In [ ]:
sw_i2d_file = 'Pipeline_Level3/full_sw_i2d.fits'
sw_sub_i2d_files = sorted(glob('Pipeline_Level3/sub[136]*_sw_i2d.fits'))

In [ ]:
sw_sub_i2d_files.remove('Pipeline_Level3/sub64p_sw_i2d.fits')
sw_sub_i2d_files

In [ ]:
sc = SourceCatalogStep.call(sw_i2d_file, save_results=True, output_dir='Pipeline_Level3', kernel_fwhm=2.0,
                            snr_threshold=50, output_file='full_sw_snr50.cat')

In [ ]:
sc = SourceCatalogStep.call(sw_sub_i2d_files[0], save_results=True, output_dir='Pipeline_Level3', kernel_fwhm=2.0,
                            snr_threshold=50, output_file='sub160_sw_cat_snr50.ecsv')

In [ ]:
sw160_snr50_catalog = ascii.read('Pipeline_Level3/sub160_sw_cat_snr50_cat.ecsv')
overlay_catalog(sw160_i2d, sw160_snr50_catalog, vmin=2, vmax=100)

In [ ]:
os.path.isfile('Pipeline_Level3/sub160_sw_cat_snr50_cat.ecsv')

In [ ]:
test_sw_sub_cat = 'Pipeline_Level3/sub160_sw_cat_snr50_cat.ecsv'
test_sw_full_cat = 'Pipeline_Level3/full_sw_snr50_cat.ecsv'

matching_thresh = 0.1  # arcsec
ee_perc = 70  # Encircled energy percentage to use. Pipeline defaults are 30, 50, 70.
limiting_vegamag = 20.

outname = 'level3_cat_compare_{}_vs_{}.txt'.format(os.path.basename(test_sw_sub_cat),
                                                   os.path.basename(test_sw_full_cat))
count_rates.compare_level3_catalogs(test_sw_full_cat, test_sw_sub_cat, outname,
                                        max_match_thresh=matching_thresh, ee_perc=ee_perc,
                                        limiting_vegamag=limiting_vegamag)

In [ ]:
sw_sub_i2d_files

In [ ]:
for sw_subfile in sw_sub_i2d_files:
    outfile = '{}_sw_cat_snr50.ecsv'.format(os.path.basename(sw_subfile).split('_')[0])
    sc = SourceCatalogStep.call(sw_subfile, save_results=True, output_dir='Pipeline_Level3', kernel_fwhm=2.0,
                            snr_threshold=50, output_file=outfile)

In [ ]:
sw_subarray_cats

In [ ]:
matching_thresh = 0.03  # arcsec
ee_perc = 70  # Encircled energy percentage to use. Pipeline defaults are 30, 50, 70.
limiting_vegamag = 20.
for sw_cat in sw_subarray_cats:
    outdir = os.path.dirname(sw_cat)
    outname = 'level3_cat_compare_{}_vs_{}.txt'.format(os.path.basename(sw_cat), os.path.basename(sw_full_cat))
    outname = os.path.join(outdir, outname)
    count_rates.compare_level3_catalogs(sw_full_cat, os.path.join(catalog_directory, sw_cat), outname,
                                        max_match_thresh=matching_thresh, ee_perc=ee_perc,
                                        limiting_vegamag=limiting_vegamag)

In [ ]:
sw_subarray_cats = sorted(glob('Pipeline_Level3/*_sw_cat_snr50*.ecsv'))

In [ ]:
sw_subarray_cats

In [ ]:
full_sw_cat_snr50 = 'Pipeline_Level3/full_sw_snr50_cat.ecsv'
matching_thresh = 0.03  # arcsec
ee_perc = 70  # Encircled energy percentage to use. Pipeline defaults are 30, 50, 70.
limiting_vegamag = 18.
for sw_cat in sw_subarray_cats:
    outdir = os.path.dirname(sw_cat)
    outname = 'level3_cat_compare_{}_vs_{}.txt'.format(os.path.basename(sw_cat),
                                                       os.path.basename(full_sw_cat_snr50))
    outname = os.path.join(outdir, outname)
    count_rates.compare_level3_catalogs(full_sw_cat_snr50,
                                        sw_cat, outname,
                                        max_match_thresh=matching_thresh, ee_perc=ee_perc,
                                        limiting_vegamag=limiting_vegamag)

In [ ]:
lw_subarray_cats

In [ ]:
matching_thresh = 0.06  # arcsec
ee_perc = 70  # Encircled energy percentage to use. Pipeline defaults are 30, 50, 70.
limiting_vegamag = 22.
for lw_cat in lw_subarray_cats:
    outdir = os.path.dirname(lw_cat)
    outname = 'level3_cat_compare_{}_vs_{}.txt'.format(os.path.basename(lw_cat), os.path.basename(lw_full_cat))
    outname = os.path.join(outdir, outname)
    count_rates.compare_level3_catalogs(lw_full_cat, os.path.join(catalog_directory, lw_cat), outname,
                                        max_match_thresh=matching_thresh, ee_perc=ee_perc,
                                        limiting_vegamag=limiting_vegamag)

In [ ]:
lw_sub_i2d_files = sorted(glob('Pipeline_Level3/sub[136]??_lw_i2d.fits'))

In [ ]:
lw_sub_i2d_files

In [ ]:
for lw_subfile in lw_sub_i2d_files:
    outfile = '{}_lw_cat_snr50.ecsv'.format(os.path.basename(lw_subfile).split('_')[0])
    sc = SourceCatalogStep.call(lw_subfile, save_results=True, output_dir='Pipeline_Level3', kernel_fwhm=2.0,
                            snr_threshold=50, output_file=outfile)

In [ ]:
outfile = 'Pipeline_Level3/full_lw_snr50_cat.ecsv'
sc = SourceCatalogStep.call('Pipeline_Level3/full_lw_i2d.fits', save_results=True, output_dir='Pipeline_Level3', kernel_fwhm=2.0,
                            snr_threshold=50, output_file=outfile)

In [ ]:
full_lw_cat_snr50 = 'Pipeline_Level3/full_lw_snr50_cat.ecsv'
matching_thresh = 0.06  # arcsec
ee_perc = 70  # Encircled energy percentage to use. Pipeline defaults are 30, 50, 70.
limiting_vegamag = 18.
for lw_cat in lw_subarray_cats:
    outdir = os.path.dirname(lw_cat)
    outname = 'level3_cat_compare_{}_vs_{}.txt'.format(os.path.basename(lw_cat),
                                                       os.path.basename(full_lw_cat_snr50))
    outname = os.path.join(outdir, outname)
    count_rates.compare_level3_catalogs(full_lw_cat_snr50,
                                        lw_cat, outname,
                                        max_match_thresh=matching_thresh, ee_perc=ee_perc,
                                        limiting_vegamag=limiting_vegamag)

In [ ]:
i2d = fits.getdata('Pipeline_Level3/full_lw_i2d.fits')
catalog = ascii.read('Pipeline_Level3/full_lw_snr50_cat.ecsv')
#overlay_catalog(i2d, catalog, vmin=2, vmax=100, start_x=836, end_x=1000,
#                start_y=836, end_y=1000)
overlay_catalog(i2d, catalog, vmin=2, vmax=100, start_x=835, end_x=995,
                start_y=879, end_y=1039)

In [ ]:
import pysiaf
n = pysiaf.Siaf('nircam')

In [ ]:
sub160_lw = n['NRCB5_SUB160']
xref = sub160_lw.XDetRef
yref = 2048 - sub160_lw.YDetRef
xmin = xref - 80
xmax = xref + 80
ymin = yref - 80
ymax = yref + 80
print(xref, yref)
print(xmin, xmax)
print(ymin, ymax)

In [ ]:
i2d = fits.getdata('Pipeline_Level3/sub160_lw_i2d.fits')
catalog = ascii.read('Pipeline_Level3/sub160_lw_cat_snr50_cat.ecsv')
overlay_catalog(i2d, catalog, vmin=2, vmax=100)